OrderBook Demo
-------------

Examples of placing orders, fill probability estimation, and simple strategy backtests

In [1]:
#Mosie Schrem
import pandas as pd
import numpy as np
import os
import math
import sys
from matplotlib import pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 10, 10
sys.path.insert(0, os.path.join(os.getcwd(), 'code'))

import OrderBook as ob
import OrderUtil as ou
import FillProbabilitySimulator as fps
import TradingStrategySimulator as tss

Examples of creating and processing orders:
--------

In [2]:
#load in intc OrderBook data
book = ob.OrderBook(message_filename='INTC_message.csv', orderbook_filename='INTC_OrderBook.csv')

Create an ask order at level 2, at time 38000, reevaluating order status at every BookUpdate 

In [3]:
order = ou.BookUpdatesOrder(orderbook=book, numupdates=1, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'queue_position': 16646, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [4]:
#process 100 updates to orderbook
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38014.459199206, 'is_buy': False, 'queue_position': 16446, 'orderstate': 'open', 'level': 2, 'current_index': 132039}


In [5]:
#process another 300 updates
for i in range(300):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38015.006473335, 'is_buy': False, 'queue_position': 16346, 'orderstate': 'open', 'level': 2, 'current_index': 132339}


In [6]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
    
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'queue_position': 0, 'orderstate': 'executed', 'level': 1, 'end_index': 134180}


We can also create orders by timestep of fixed size

Create an ask order at level 2, at time 38000, reevaluating order status every 0.5 seconds

In [7]:
order = ou.TimeOrder(orderbook=book, delta_t=0.5, timestamp=38000, level=2, is_buy=False)
print(order.get_opening_stats())

{'price': 273800, 'start_time': 38000, 'is_buy': False, 'queue_position': 16646, 'orderstate': 'open', 'level': 2, 'start_index': 131939}


In [8]:
#process 50 seconds of data
for i in range(100):
    order.update()
print(order.get_current_stats())

{'price': 273800, 'time': 38049.987429576, 'is_buy': False, 'queue_position': 13946, 'orderstate': 'open', 'level': 3, 'current_index': 132820}


In [9]:
#process until executed or cancelled
while(order.order_type('open')):
    order.update()
#print stats at time when order was executed
print(order.get_closing_stats())

{'price': 273800, 'time': 38167.621597535, 'is_buy': False, 'queue_position': 0, 'orderstate': 'executed', 'level': 1, 'end_index': 134180}


Fill Probability Simulation Example:
------

In the following example, we quote buy and sell orders in the inside market, and reevaluate every 100 BookUpdates

In [10]:
probability_simulator = fps.FillProbabilitySimulator(numupdates=100, order_tuple=([1] , [1]),
                                                 orderbook=book, t_start=34200.01, t_end=50000)
probability_simulator.generate_matrices(num_samples=10000)

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0


Display empirical fill probabilities for given midprice movement -1, 0, and 1

Note that bid_level_i_ask_level_j refers to bid orders filled up to level i and ask orders filled up to level j
bid_level_0 (ask_level_0) refers to no bids (asks) filled.

In [11]:
fill_matrix = probability_simulator.get_prob_matrix()
fill_matrix

y,-1,0,1
orders_executed,,,
bid_level_0_ask_level_0,0.387223,0.940594,0.373537
bid_level_0_ask_level_1,0.000000,0.030303,0.621662
bid_level_1_ask_level_0,0.599580,0.027303,0.000600
bid_level_1_ask_level_1,0.013197,0.001800,0.004200


Display frequencies of midprice movements in our sample

In [12]:
print(probability_simulator.get_quantities())

{-1: 3334, 0: 3333, 1: 3333}


Here we display the cumulative distribution of our fill probability matrix

In [13]:
cum_mat = probability_simulator.get_cum_prob_matrix()
cum_mat

y,-1,0,1
orders_executed,,,
bid_level_0_ask_level_0,1.000000,1.000000,1.000000
bid_level_0_ask_level_1,0.013197,0.032103,0.625863
bid_level_1_ask_level_0,0.612777,0.029103,0.004800
bid_level_1_ask_level_1,0.013197,0.001800,0.004200


Display PNLs

In [14]:
pnls = probability_simulator.get_pnls()
pnls

y,-1,0,1
orders placed,,,
"bids: [1], asks: [1]",-19.91,0.06,-20.69


Loss Function Estimation
-------------

Here we approximate our loss function to feed into our Neural Network using a similar approach to the fill probability simulation.

We use the same strategy as we did for the above backtest example

In [15]:
strategy = {1: ([1], [2]), 0: ([1], [1]), -1 : ([2], [1])}
loss_func = fps.LossFunction(strategy=strategy, book=book, numupdates=100, t_end=50000, uniform_sampling=True)
loss_func.generate_loss_function_and_fill_probabilities(num_samples=10000)

simulating strategy for case yhat = 1...

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0
simulating strategy for case yhat = 0...

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples processed so far: 7000.0
samples processed so far: 8000.0
samples processed so far: 9000.0
samples processed so far: 10000.0
simulating strategy for case yhat = -1...

samples processed so far: 1000.0
samples processed so far: 2000.0
samples processed so far: 3000.0
samples processed so far: 4000.0
samples processed so far: 5000.0
samples processed so far: 6000.0
samples proce

Loss Matrix Below based on average PNL in given case (-loss are cases where our strategy yields a profit)

In [16]:
loss_function_matrix = loss_func.get_loss_matrix()
loss_function_matrix.head()

y_true:,-1,0,1
y_predicted,,,
-1,0.17,-0,21.18
0,20.12,-0.03,20.35
1,20.77,-0.01,0.1


Fill Probabilities for each strategy

In [17]:
for key in strategy:
    print("\nFill probabilities for yhat: " + str(key) + ", and trade: " + str(strategy[key]))
    print(loss_func.get_fill_probabilities()[key])


Fill probabilities for yhat: 1, and trade: ([1], [2])
y                              -1         0         1
orders_executed                                      
bid_level_0_ask_level_0  0.388122  0.968497  0.969997
bid_level_0_ask_level_2  0.000000  0.000300  0.021902
bid_level_1_ask_level_0  0.611878  0.031203  0.003900
bid_level_1_ask_level_2  0.000000  0.000000  0.004200

Fill probabilities for yhat: 0, and trade: ([1], [1])
y                              -1         0         1
orders_executed                                      
bid_level_0_ask_level_0  0.386323  0.945095  0.381338
bid_level_0_ask_level_1  0.000000  0.027003  0.612961
bid_level_1_ask_level_0  0.602579  0.027003  0.001500
bid_level_1_ask_level_1  0.011098  0.000900  0.004200

Fill probabilities for yhat: -1, and trade: ([2], [1])
y                              -1         0         1
orders_executed                                      
bid_level_0_ask_level_0  0.968506  0.967597  0.368437
bid_level_0_ask_level_1 